In [2]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import os
import re

import glob
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.impute import KNNImputer

%autosave 120

Autosaving every 120 seconds


## 實價登錄資料: 1_data

In [2]:
path_ = r'/Users/liang/Documents/NCKU_DS/資料科學/數位沙盒/raw_data'
data1 = pd.read_csv(path_ + r'/AAPRTRAN_20171201_20191231.csv')
data2 = pd.read_csv(path_ + r'/AAPRTRAN_20200101_20220101.csv')
data3 = pd.read_csv(path_ + r'/AAPRTRAN_after_20220101.csv')
data = pd.concat([data1, data2, data3], axis=0)
print(data1.shape)
print(data2.shape)
print(data3.shape)
print(data.shape)
#data

(758223, 54)
(843462, 54)
(340936, 54)
(1942621, 54)


In [3]:
data = data[data['location'].str.contains('臺北|台北')==True]
data = data[data['district_id'].str.contains('瑞芳區')==False]
print(data.shape)

(70823, 54)


In [4]:
# Duplicated
print(data.duplicated().value_counts())
data[data.duplicated()]

False    70823
Name: count, dtype: int64


,None,coid,mdate,key3,district_id,district,tran_tp,tran_tp_fg,tsign,tsign_fg,...,berth_tp_fg,berth_area,tot_prc,unit_prc,berth_prc,rmk,ann_date,abnorm_td,abnorm_loc,abnorm_prc


In [5]:
data = data.rename(columns={data.columns[0]: 'ID'})
print('原始資料：遺失值之變數及數量')
print(data.isna().sum()[data.isna().sum()!=0])

原始資料：遺失值之變數及數量
road_fg         6927
coord_x         7532
coord_y         7532
zoning           783
zoning_fg        783
ncity_z        70690
ncity_z_fg     70690
land_use       70823
land_use_fg    70823
tran_floor       719
mainuse         2375
mainuse_fg      2375
matl             729
matl_fg          729
build_dd       12602
room_age       15693
berth_tp       42463
berth_tp_fg    42463
rmk            32646
abnorm_td      70823
abnorm_loc     70204
abnorm_prc     70787
dtype: int64


In [6]:
data = data[~data['tsign'].isin(['土地', '車位'])]
data.shape

(67365, 54)

In [7]:
# print('有遺失值之變數')
# print(data.isna().sum()[data.isna().sum()!=0])

In [8]:
# 去掉不必要欄位
data = data.drop(['ncity_z', 'ncity_z_fg', 'land_use', 'land_use_fg', 'abnorm_td', 'abnorm_loc', 'abnorm_prc'], axis=1)
data = data.drop(['coid', 'key3', 'district'], axis=1) #縣市、案碼
data.shape

(67365, 44)

In [9]:
# 去掉時間多餘的部分
data['mdate'] = pd.to_datetime(data['mdate']).dt.date
data['ann_date'] = pd.to_datetime(data['ann_date']).dt.date
data['build_dd'] = pd.to_datetime(data['build_dd']).dt.date

In [10]:
# 交易日 - 建築完成日期 當作屋齡，若為負則補 0
time_delta = pd.to_timedelta(data['mdate'] - data['build_dd'], errors='coerce')
age = time_delta.dt.days/365
age = age.clip(lower=0)  #若小於０ 則算做0

data['room_age']= data['room_age'].fillna(age) #使用 age 的值填充原data['room_age']中的 NaN

In [11]:
# 刪除為分類註記的變數（多餘的）
data = data.drop(['tran_tp_fg','tsign_fg','zoning_fg','mainuse_fg', 'berth_tp_fg', 'matl_fg', 'bstate_fg'], axis=1) #縣市、案碼
data.shape

(67365, 37)

In [12]:
# 若備註欄位出現 ‘預售屋’ 則 tran_tp (交易別) 改為預售屋
data.loc[data.index[data['rmk'].str.contains('預售')==True], 'tran_tp'] = '預售屋'

# 若備註欄位出現 ‘毛胚屋’ 則 tran_tp (交易別) 改為毛胚屋
data.loc[data.index[data['rmk'].str.contains('毛胚')==True], 'tran_tp'] = '毛胚屋'

In [13]:
# 若 berth_prc(車位總價)>0但 berth_area(車位總面積)為 0 的話, 則將 berth_area 改為 NA (原資料直接給0...)
data.loc[(data['berth_prc'] > 0) & (data['berth_area'] == 0), 'berth_area'] = np.nan

In [14]:
# 將房廳衛合併為類別變數
# layout = data['room'].astype(str) + '房' + data['hall'].astype(str) + '廳' + data['health'].astype(str) + '衛'
# data['layout'] = layout

In [15]:
print('有遺失值之變數')
print(data.isna().sum()[data.isna().sum()!=0])

有遺失值之變數
road_fg        6849
coord_x        7452
coord_y        7452
zoning          644
tran_floor       11
mainuse        1646
matl             21
build_dd      11808
room_age      11808
berth_tp      42447
berth_area     1044
rmk           31035
dtype: int64


In [16]:
# 匯出資料
path_clean = r'/Users/liang/Documents/NCKU_DS/資料科學/數位沙盒/clean_data'
data.to_csv(path_clean + r'/1_data.csv', sep=',', encoding='utf-8', index=False)

## 實價登錄資料: 2_data

In [28]:
path_clean = r'/Users/liang/Documents/NCKU_DS/資料科學/數位沙盒/clean_data'
data = pd.read_csv(path_clean+ r'/1_data.csv')
print(data.shape)

(67365, 37)


In [29]:
# 函數來將中文數字轉換為阿拉伯數字
def chinese_to_arabic(cn_num):
    chinese_numbers = {
        '一': 1, '二': 2, '兩': 2, '三': 3, '四': 4, '五': 5,
        '六': 6, '七': 7, '八': 8, '九': 9, '十': 10
    }
    total = 0
    r = 1  # 表示單位：十、百、千...

    for i in range(len(cn_num) - 1, -1, -1):
        val = chinese_numbers.get(cn_num[i])
        if val is not None:
            if val == 10:
                if i == 0:
                    total += 10
                else:
                    r = 10
            else:
                total += val * r
                r *= 10
    return total

# 函數來解析樓層資料
def parse_floor(floor):
    if pd.isna(floor):
        return [np.nan, np.nan]
    # 使用正則表達式找出所有中文數字
    if floor == '全':
        return[0, np.nan]
    floors = re.findall(r'[一二兩三四五六七八九十]+', floor)
    floors = [chinese_to_arabic(f) for f in floors]
    if len(floors) == 0:
        return [np.nan, np.nan]
    return [min(floors), max(floors)]

In [30]:
floors = data['tran_floor'].tolist()

parsed_floors = [parse_floor(floor) for floor in floors]
floor_df = pd.DataFrame(parsed_floors, columns=['最低樓層', '最高樓層'])

data = pd.concat([data, floor_df], axis=1)
data.loc[data['最低樓層'] == 0, '最高樓層'] = data['tot_floor']
data = data.drop(["tran_floor", "tran_up_ll", "tran_single"], axis=1)
data.shape

(67365, 36)

In [31]:
# build_dd建造日期, compart不動產有無隔間, ann_date 公告日 刪
data = data.drop(['build_dd', 'compart', 'ann_date'], axis=1)
data.shape

(67365, 33)

In [38]:
# 特殊關係交易：刪
data = data[data['rmk'].str.contains('特殊關係')!=True]
print(data.shape)

# 交易標的為建物：刪
data = data[data['tsign'] != '建物']
print(data.shape)

# 建物型態: 倉庫工廠單價都是0 刪
data = data[data['bstate']!='倉庫']
data = data[data['bstate']!='工廠']
print(data.shape)

# 主要用途： 住商 工業 辦公室 留 其他併 其他
data.loc[~data['mainuse'].isin(['住宅', '商業用', '住商用', '工業用', '辦公室']), 'mainuse'] = '其他'
print(data.shape)

(60923, 33)
(60923, 33)
(60923, 33)
(60923, 33)


In [39]:
print('有遺失值之變數')
print(data.isna().sum()[data.isna().sum()!=0])

有遺失值之變數
road_fg        6634
coord_x        7191
coord_y        7191
zoning          206
matl             19
room_age      10540
berth_tp      37147
berth_area     1012
rmk           30890
最低樓層            161
最高樓層            161
dtype: int64


In [36]:
# 匯出資料
path_clean = r'/Users/liang/Documents/NCKU_DS/資料科學/數位沙盒/clean_data'
data.to_csv(path_clean + r'/2_data.csv', sep=',', encoding='utf-8', index=False)

### 實價登錄資料: 3_data


In [4]:
path_clean = r'/Users/liang/Documents/NCKU_DS/資料科學/數位沙盒/clean_data'
data = pd.read_csv(path_clean+ r'/2_data.csv')
print(data.shape)

(60923, 33)


In [42]:
# 刪除 rmk tot_prc
data = data.drop(['rmk', 'tot_prc'], axis=1)
data.shape

(60923, 31)

In [45]:
data.columns = ['ID', '交易日', '鄉鎮市區', '交易別', '交易標的', '門牌/地號', '臨路(Y/N)',
       '緯度', '經度', '土地', '建物', '車位', '土地移轉坪數', '都市土地使用分區',
       '總樓層', '頂樓註記(Y/N)', '建物型態', '主要用途', '主要建材', '屋齡(年)',
       '建物移轉坪數', '格局-房數', '格局-廳數', '格局-衛浴數', '管理組織(Y/N)', '車位類別',
       '車位總面積(坪)', '單價(萬/坪)', '	車位總價(萬)', '最低樓層', '最高樓層']

In [47]:
print('有遺失值之變數')
print(data.isna().sum()[data.isna().sum()!=0])

有遺失值之變數
臨路(Y/N)      6634
緯度           7191
經度           7191
都市土地使用分區      206
主要建材           19
屋齡(年)       10540
車位類別        37147
車位總面積(坪)     1012
最低樓層          161
最高樓層          161
dtype: int64


In [46]:
# 匯出資料
data.to_csv(path_clean + r'/3_data.csv', sep=',', encoding='utf-8', index=False)

In [ ]:
# 經緯度有 7532 個遺失值

### 補完經緯度資料

In [3]:
path_ = r'/Users/liang/Documents/NCKU_DS/資料科學/數位沙盒/clean_data'
data6 = pd.read_csv(path_ + r'/6_data.csv')


In [10]:
data6['單價(萬/坪)'].describe()

count    57028.000000
mean        71.240126
std         29.754180
min          0.000000
25%         52.200000
50%         66.200000
75%         83.700000
max        713.500000
Name: 單價(萬/坪), dtype: float64

In [14]:
tryy = data6[data6['單價(萬/坪)']==0]
tryy.to_csv(path_ + r'/tryy.csv', sep=',', encoding='utf-8', index=False)

### 7_data
- knnimputer 補遺失值
- 刪除車位類別變數
- 刪除地址欄位
- 處理類別變數:oh, homals


In [24]:
path_ = r'/Users/liang/Documents/NCKU_DS/資料科學/數位沙盒/clean_data'
data6 = pd.read_csv(path_ + r'/6_data.csv')
data6.shape

(57028, 49)

In [25]:
# 刪除車位類別變數
# 刪除地址欄位
data6 = data6.drop(['門牌/地號', '車位類別'], axis=1)
data6.shape

(57028, 47)

In [34]:
print(data6.isna().sum()[data6.isna().sum()!=0])

臨路(Y/N)      6613
都市土地使用分區      191
主要建材           19
屋齡(年)       10489
車位總面積(坪)     1012
最低樓層          158
最高樓層          158
dtype: int64


In [27]:
# 將日期直接轉為自Unix纪元（1970年1月1日）以來的秒數
data6['交易日'] = data6['交易日'].apply(lambda x: pd.to_datetime(x).timestamp())
data6.shape

(57028, 47)

In [30]:
# 連續
data6_con = data6._get_numeric_data()
print(data6_con.shape)
print(data6_con.columns)

# 類別
data6_ca = data6.loc[:, data6.columns.difference(data6_con.columns)]
print(data6_ca.shape)
print(data6_ca.columns)

(57028, 37)
Index(['ID', '交易日', '緯度', '經度', '土地', '建物', '車位', '土地移轉坪數', '總樓層', '屋齡(年)',
       '建物移轉坪數', '格局-房數', '格局-廳數', '格局-衛浴數', '車位總面積(坪)', '單價(萬/坪)', '車位總價(萬)',
       '最低樓層', '最高樓層', 'conven_500', 'bus_500', 'dental_1000', 'clinic_1000',
       '中醫_1000', 'nearest_mrt', 'nearest_hospital', 'nearest_conven',
       'nearest_中醫', 'nearest_bus', 'nearest_junior', 'nearest_university',
       'nearest_elementary', 'nearest_dental', 'nearest_clinic',
       'nearest_postoffice', 'nearest_bank', 'nearest_high'],
      dtype='object')
(57028, 10)
Index(['主要建材', '主要用途', '交易別', '交易標的', '建物型態', '管理組織(Y/N)', '臨路(Y/N)',
       '都市土地使用分區', '鄉鎮市區', '頂樓註記(Y/N)'],
      dtype='object')


In [36]:
data6_con_ca = pd.concat([data6_con, data6_ca], axis = 1)
data6_con_ca.shape

(57028, 47)

In [33]:
# 先做 KNNImputer
# imputer = KNNImputer(n_neighbors=10, weights = 'distance')
# data6_knnimpute = imputer.fit_transform(np.array(data6_con_ca))
# data6_final = pd.DataFrame(data6_knnimpute, columns=o)

In [76]:
# data6_ca

In [7]:
# 將類別變數做oh
data6_ca_encoded = pd.get_dummies(data6_ca, columns=data6_ca.columns)
print('one hot encoding', data6_ca_encoded.shape)

# 將類別變數匯出 再r上做homals
# data6_ca.to_csv(path_ + r'/data6_ca.csv', sep=',', encoding='utf-8', index=False)

one hot encoding (57028, 69)


In [92]:
# r 做的homals
# homals = pd.read_csv(path_ + r'/homals.csv')
# print('homals', homals.shape)

homals (57028, 10)


In [19]:
# 將連續變數與類別變數合併

# oh
data6_oh = pd.concat([data6_con, data6_ca_encoded], axis = 1)
print(data6_oh.shape)

# homals
# data6_homals = pd.concat([data6_con, homals], axis = 1)
# print(data6_homals.shape)

(57028, 106)


In [20]:
o = data6_oh.columns
# h = data6_homals.columns

In [21]:
print(data6_oh.isna().sum()[data6_oh.isna().sum()!=0])

屋齡(年)       10489
車位總面積(坪)     1012
最低樓層          158
最高樓層          158
dtype: int64


In [104]:
# knn imputer

# oh
imputer = KNNImputer(n_neighbors=10, weights = 'distance')
oh_knnimpute = imputer.fit_transform(np.array(data6_oh))

# homals
# imputer = KNNImputer(n_neighbors=10, weights = 'distance')
# homals_knnimpute = imputer.fit_transform(np.array(data6_homals))

In [107]:
# data_homals = pd.DataFrame(homals_knnimpute, columns=h)

In [112]:
print(data_oh.shape)
# print(data_homals.shape)

(57028, 106)
(57028, 47)


In [109]:
data_oh.to_csv(path_ + r'/7_data_oh.csv', sep=',', encoding='utf-8', index=False)
# data_homals.to_csv(path_ + r'/7_data_homals.csv', sep=',', encoding='utf-8', index=False)

### 11_data
刪除單價為0

In [17]:
path_ = r'/Users/liang/Documents/NCKU_DS/資料科學/數位沙盒/clean_data'
data10 = pd.read_csv(path_ + r'/10_data.csv')

In [18]:
data10.head()

,ID,交易日,緯度,經度,土地,建物,車位,土地移轉坪數,總樓層,屋齡(年),...,單價_16,單價_17,單價_18,單價_19,單價_20,單價_21,單價_22,單價_23,單價_24,單價(萬/坪)
0,0.794471,0.0,0.391260,0.416567,0.000000,0.02439,0.026316,0.002585,0.309524,0.002879,...,0.045777,0.04419,0.059085,0.069863,0.027628,0.053462,0.065563,0.021953,0.045576,115.8
1,0.794495,0.0,0.467469,0.927570,0.000000,0.02439,0.052632,0.004234,0.333333,0.100768,...,0.045777,0.04419,0.059085,0.069863,0.027628,0.053462,0.065563,0.021953,0.045576,51.9
2,0.794518,0.0,0.359303,0.264357,0.000000,0.02439,0.000000,0.001640,0.571429,0.001919,...,0.045777,0.04419,0.059085,0.069863,0.027628,0.053462,0.065563,0.021953,0.045576,59.3
3,0.794541,0.0,0.530375,0.329790,0.000000,0.02439,0.052632,0.009023,0.261905,0.119002,...,0.045777,0.04419,0.059085,0.069863,0.027628,0.053462,0.065563,0.021953,0.045576,50.2
4,0.794565,0.0,0.904310,0.540819,0.038462,0.02439,0.000000,0.369285,0.023810,0.564299,...,0.045777,0.04419,0.059085,0.069863,0.027628,0.053462,0.065563,0.021953,0.045576,534.7


In [21]:
data11 = data10[data10['單價(萬/坪)']!=0]

In [23]:
data11.shape

(33142, 190)

In [24]:
data11.to_csv(path_ + r'/11_data.csv', sep=',', encoding='utf-8', index=False)

### 12_data 按時間切 train test

In [12]:
path_ = r'/Users/liang/Documents/NCKU_DS/資料科學/數位沙盒/clean_data'
data8 = pd.read_csv(path_ + r'/8_data.csv')

In [34]:
data8_11 = data8[data8['單價(萬/坪)']!=0]

In [73]:
new = pd.to_datetime(data8_11['交易日'], unit='s')[data8_11.shape[0]-33142:]
new.to_csv(path_ + r'/11_data_date.csv', sep=',', encoding='utf-8', index=False)
new

23875   2019-12-01
23876   2019-12-01
23877   2019-12-01
23878   2019-12-01
23879   2019-12-01
           ...    
57023   2022-12-31
57024   2022-12-31
57025   2022-12-31
57026   2022-12-31
57027   2022-12-31
Name: 交易日, Length: 33142, dtype: datetime64[ns]

In [62]:
new[:-6603]

23875   2019-12-01
23876   2019-12-01
23877   2019-12-01
23878   2019-12-01
23879   2019-12-01
           ...    
50415   2022-06-15
50416   2022-06-15
50417   2022-06-15
50418   2022-06-15
50419   2022-06-15
Name: 交易日, Length: 26539, dtype: datetime64[ns]

In [63]:
new[-6603:]

50420   2022-06-16
50421   2022-06-16
50422   2022-06-16
50423   2022-06-16
50424   2022-06-16
           ...    
57023   2022-12-31
57024   2022-12-31
57025   2022-12-31
57026   2022-12-31
57027   2022-12-31
Name: 交易日, Length: 6603, dtype: datetime64[ns]

In [64]:
print(26539/33142)
print(6603/33142)

0.800766399131012
0.199233600868988


In [66]:
path_ = r'/Users/liang/Documents/NCKU_DS/資料科學/數位沙盒/clean_data'
data11 = pd.read_csv(path_ + r'/11_data.csv')
data11.shape

(33142, 190)

In [72]:
data12_train = data11.iloc[:26539, ] # 2022-06-15以後
data12_test = data11.iloc[26539:, ]

data12_train.to_csv(path_ + r'/12_data_train.csv', sep=',', encoding='utf-8', index=False)
data12_test.to_csv(path_ + r'/12_data_test.csv', sep=',', encoding='utf-8', index=False)

In [3]:
path_ = r'/Users/liang/Documents/NCKU_DS/資料科學/數位沙盒/clean_data'
data12_train = pd.read_csv(path_ + r'/12_data_train.csv')
data12_test = pd.read_csv(path_ + r'/12_data_test.csv')

print(data12_train.shape)
print(data12_test.shape)

(26539, 190)
(6603, 190)


In [7]:
print(data12_train.isna().sum())#[data12_train.isna().sum()!=0])
print(data12_test.isna().sum())#[data12_train.isna().sum()!=0])

ID         0
交易日        0
緯度         0
經度         0
土地         0
          ..
單價_21      0
單價_22      0
單價_23      0
單價_24      0
單價(萬/坪)    0
Length: 190, dtype: int64
ID         0
交易日        0
緯度         0
經度         0
土地         0
          ..
單價_21      0
單價_22      0
單價_23      0
單價_24      0
單價(萬/坪)    0
Length: 190, dtype: int64
